In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 17.3 MB/s 
     |████████████████████████████████| 7.6 MB 5.8 MB/s 
     |████████████████████████████████| 182 kB 59.5 MB/s 


In [2]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext.data.metrics import bleu_score

from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random

from tqdm.auto import tqdm

In [3]:
data_file_open = open('/content/data_f.txt', 'r', encoding='UTF-8')
label_file_open = open('/content/label_f.txt', 'r', encoding='UTF-8')
data_lines=data_file_open.readlines()
label_lines = label_file_open.readlines()

input_dataset = []
output_dataset = []
dataset = []


for item in data_lines:
    input_dataset.append(item.strip())
for item in label_lines:
    output_dataset.append(item.strip())

In [4]:
print(len(input_dataset))
print(len(output_dataset))

42535
42535


In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)

tokenizer.pad_token = tokenizer.eos_token

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [6]:
dataset = []
for i in range(len(input_dataset)):
    data = input_dataset[i]+' = '+output_dataset[i]
    #if(len(data)>200):
    #    continue
    #if(len(data)<20):
    #    continue
    dataset.append(data)

In [7]:
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2)

In [8]:
print(len(train_dataset))
print(len(val_dataset))

34028
8507


In [9]:
tokenized_dataset_train = []

tokenized_dataset_val = []


for data in train_dataset:
    tokenized_dataset_train.append(tokenizer.encode(data))

for data in val_dataset:
    tokenized_dataset_val.append(tokenizer.encode(data))

In [10]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [13]:
print(model.named_parameters())

<generator object Module.named_parameters at 0x7f248b80b550>


In [21]:
for name, param in model.named_parameters():
    if name.startswith("transformer.h.10"):
        param.requires_grad = True
    elif name.startswith("transformer.h.11"):
        param.requires_grad = True
    elif name.startswith("transformer.ln_f"):
        param.requires_grad = True
    else:
        param.requires_grad = False

In [29]:
for name, param in model.named_parameters():
    if name.startswith("transformer.ln_f"):
        print(name)
        print(param.requires_grad)

transformer.ln_f.weight
True
transformer.ln_f.bias
True


In [33]:
training_args = TrainingArguments(
          output_dir='/content/train_2',
          learning_rate=2e-5,
          per_device_train_batch_size=8,
          per_device_eval_batch_size=8,
          num_train_epochs=3,
          evaluation_strategy = "epoch",
          save_strategy='epoch'
      )

In [36]:
trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=tokenized_dataset_train,
          eval_dataset=tokenized_dataset_val,
          data_collator=data_collator)

In [37]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 34028
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12762
  Number of trainable parameters = 14177280


Step,Training Loss
500,3.884700
1000,3.685700
1500,3.617200
2000,3.543000
2500,3.508500
3000,3.513400
3500,3.503800
4000,3.468900
4500,3.427500
5000,3.399100


Saving model checkpoint to /content/train_2/checkpoint-4254
Configuration saved in /content/train_2/checkpoint-4254/config.json
Model weights saved in /content/train_2/checkpoint-4254/pytorch_model.bin
Saving model checkpoint to /content/train_2/checkpoint-8508
Configuration saved in /content/train_2/checkpoint-8508/config.json
Model weights saved in /content/train_2/checkpoint-8508/pytorch_model.bin
Saving model checkpoint to /content/train_2/checkpoint-12762
Configuration saved in /content/train_2/checkpoint-12762/config.json
Model weights saved in /content/train_2/checkpoint-12762/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=12762, training_loss=3.4275518051790717, metrics={'train_runtime': 861.603, 'train_samples_per_second': 118.481, 'train_steps_per_second': 14.812, 'total_flos': 2706532273152000.0, 'train_loss': 3.4275518051790717, 'epoch': 3.0})

In [38]:
trainer.save_model()

Saving model checkpoint to /content/train_2
Configuration saved in /content/train_2/config.json
Model weights saved in /content/train_2/pytorch_model.bin


In [39]:
model = GPT2LMHeadModel.from_pretrained('/content/train_2')

loading configuration file /content/train_2/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_siz

In [65]:
train_dataset[0]

"Let's see, will his finger catch fire? = Come, will this wood take fire?"

In [87]:
val_dataset[99]

"He can't be sending you to murder me for the prince's death  because in that, he is just as guilty as I am. = He sends you not to murder me for this, For in that sin he is as deep as I."

In [ ]:
input = tokenizer.encode(input_dataset[0]+' = ')

In [90]:
input = tokenizer.encode('Hello, how are you? = ')

output = model.generate(input_ids=torch.tensor([input]),max_new_tokens=30, do_sample=True)

tokenizer.decode(output[0])

'Hello, how are you? =  Thither, how are you? How now.  Why now? = Wherefore, how now? Wherefore? What now? = How now'

In [44]:
input = tokenizer.encode('You were either ignorant of it or, seeing it, why were you so childishly friendly? = ')

output = model.generate(input_ids=torch.tensor([input]),max_new_tokens=30, do_sample=True)

tokenizer.decode(output[0])

'You were either ignorant of it or, seeing it, why were you so childishly friendly? =  And, if you were ignorant of it, why were you so childishly kindly? Were you ignorant, or, seeing it, were thou so child'

In [84]:
input = tokenizer.encode('I will always follow your instructions. = ')

output = model.generate(input_ids=torch.tensor([input]),max_new_tokens=30, do_sample=True)

tokenizer.decode(output[0])

"I will always follow your instructions. =  I'll always follow thy advice. = I will always follow thy will. I’ll follow thy instructions.  I’ll follow thy"

In [83]:
input = tokenizer.encode('We shall leave tomorrow. = ')

output = model.generate(input_ids=torch.tensor([input]),max_new_tokens=30, do_sample=True)

tokenizer.decode(output[0])

'We shall leave tomorrow. =  As you, I will leave tomorrow. (Pause. = And thus this final moment. = We shall leave. Then. Now. Here. Now'

In [68]:
input = tokenizer.encode('Let\'s see, will his finger catch fire? = ')

output = model.generate(input_ids=torch.tensor([input]),max_new_tokens=30, do_sample=True)

tokenizer.decode(output[0])

"Let's see, will his finger catch fire? =  'Tis his finger, but will he set it free soon?  'Tis, his palm will do soon? = Why, is his"

In [89]:
input = tokenizer.encode('He can\'t be sending you to murder me for the prince\'s death  because in that, he is just as guilty as I am. = ')

output = model.generate(input_ids=torch.tensor([input]),max_new_tokens=30, do_sample=True)

tokenizer.decode(output[0])

'He can\'t be sending you to murder me for the prince\'s death  because in that, he is just as guilty as I am. =  "If he had sent me to the prince For doing so would render him dead.  \'Tis the prince\'s death that should be the reason'

In [ ]:
model = GPT2LMHeadModel.from_pretrained('/content/')

loading configuration file /content/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_size": 5025

OSError: ignored

In [ ]:
from google.colab import drive

import shutil

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
shutil.copy("/content/train_1/config.json","/content/drive/MyDrive/ECE1786_Project/Models/GPT2_Trainer_2/")
shutil.copy("/content/train_1/pytorch_model.bin","/content/drive/MyDrive/ECE1786_Project/Models/GPT2_Trainer_2/")
shutil.copy("/content/train_1/training_args.bin","/content/drive/MyDrive/ECE1786_Project/Models/GPT2_Trainer_2/")

'/content/drive/MyDrive/ECE1786_Project/Models/GPT2_Trainer_1/training_args.bin'